# Worker1
running another notebook

In [ ]:
""" Setup
mounting the drive and grabbing main
"""

# from "copy path" in sagemaker studio file browser
main_file = 'NonHomogeneousModeling/09_SagemakerStudioPort/09_XGBHypeStudy.ipynb'
main_file = '/root/' + main_file

import os

with open(main_file, 'r') as f:
  code = f.read()

""" Running
Parsing the content to get all code cells, and running them sequentially.
"""

import json
import nbformat

for cell in nbformat.read(main_file, as_version=4).cells:
    print(cell)
    if cell['cell_type'] == 'code':
        if len(cell['source']):
            src = cell['source']
            if src[0] == '!':
                print('============= running command "{}" =============='.format(src[1:]))
                os.system(src[1:])
            else:
                try:
                    exec(src)
                except Exception as e:
                    print('---------')
                    print(src)
                    print('---------')
                    raise(e)

{'cell_type': 'markdown', 'metadata': {'id': 'LCq0El6rCixe'}, 'source': '# 09 XGB Hype Study\nPerforming a hyperparameter study on XGBoost to understand which hyperparameters have the biggest impact on performance. This notebook runs a whole lot of runs, which can then be analyzed on a parameter by parameter basis.\n\nenv: Data Science 3.0\n\ninstance: ml.t3.medium'}
{'cell_type': 'markdown', 'metadata': {'id': 'bv29yMJmFa0W'}, 'source': '# installing the most up-to-date experiment tracker client\nand dependencies'}
{'cell_type': 'code', 'execution_count': 2, 'metadata': {'colab': {'base_uri': 'https://localhost:8080/'}, 'executionInfo': {'elapsed': 30877, 'status': 'ok', 'timestamp': 1695827063534, 'user': {'displayName': 'Daniel Warfield', 'userId': '15280522968339009252'}, 'user_tz': 300}, 'id': 'HIlfPmIaCgsO', 'outputId': '007fb8dc-52c0-4ec8-a42c-ad817434076c', 'scrolled': True, 'tags': []}, 'outputs': [{'name': 'stdout', 'output_type': 'stream', 'text': "Collecting git+https://git

  Running command git clone --filter=blob:none --quiet https://github.com/DanielWarfield1/TabularExperimentTrackerClient /tmp/pip-req-build-xirbgroo


  Resolved https://github.com/DanielWarfield1/TabularExperimentTrackerClient to commit 5e738443d55b9637454776bd740a8083af70272d
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


{'cell_type': 'code', 'execution_count': 3, 'metadata': {'tags': []}, 'outputs': [], 'source': '#grabbing credentials from seceret manager\nwith open ("/root/Credentials/openMLAPIKey.txt", "r") as myfile:\n    opml_key = myfile.read()\nwith open ("/root/Credentials/tabExpTrackAPIKey.txt", "r") as myfile:\n    orch_key = myfile.read()'}
{'cell_type': 'code', 'execution_count': 4, 'metadata': {'colab': {'base_uri': 'https://localhost:8080/'}, 'executionInfo': {'elapsed': 52110, 'status': 'ok', 'timestamp': 1695827115624, 'user': {'displayName': 'Daniel Warfield', 'userId': '15280522968339009252'}, 'user_tz': 300}, 'id': 'TcQIEIk-xLBy', 'outputId': 'eb82b8ad-a3e6-49bb-9c8e-e2c77de353f4', 'tags': []}, 'outputs': [], 'source': "#configuring the experiment tracker\nfrom TabularExperimentTrackerClient.ExperimentClient import ExperimentClient\n\n#creating experiment client utilities\nex = ExperimentClient(verbose = False)\n#getting openml credentials from drive\nex.define_opml_cred(opml_key)\n

{'cell_type': 'markdown', 'metadata': {'id': 'j6qihLeaGIAw'}, 'source': '# Defining Hyperparameter Spaces\nusing hyperparameter space defined in https://arxiv.org/pdf/2207.08815.pdf\n\nall hyperparameter spaces respect the naming convention of the module they will be used with, for convenience sake.'}
{'cell_type': 'code', 'execution_count': 9, 'metadata': {'executionInfo': {'elapsed': 190, 'status': 'ok', 'timestamp': 1695831425417, 'user': {'displayName': 'Daniel Warfield', 'userId': '15280522968339009252'}, 'user_tz': 300}, 'id': '4ahRltMjF55F', 'tags': []}, 'outputs': [], 'source': 'import copy\nimport numpy as np\n\n#============================================================\n# XGBoost\n#============================================================\n\n# for https://xgboost.readthedocs.io/en/stable/parameter.html\nXGBoost_space = {\n    "max_depth": {"distribution": "int_uniform", "min":1, "max":11},\n    "n_estimators": {"distribution": "int_uniform", "min":100, "max":200},\n    

  Running command git clone --filter=blob:none --quiet https://github.com/Bartosz-G/NeuralNetworksTrainingPackage /tmp/pip-req-build-y8respek


  Resolved https://github.com/Bartosz-G/NeuralNetworksTrainingPackage to commit 2eff78e8c87ac0bc780226edc9f1b16ac8629ca6
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


{'cell_type': 'code', 'execution_count': 17, 'metadata': {'executionInfo': {'elapsed': 11, 'status': 'ok', 'timestamp': 1695831458391, 'user': {'displayName': 'Daniel Warfield', 'userId': '15280522968339009252'}, 'user_tz': 300}, 'id': 'ygSeifgSmO1H', 'tags': []}, 'outputs': [], 'source': "\nfrom NeuralNetworksTrainingPackage.event_handler import dataPreProcessingEventEmitter\nfrom NeuralNetworksTrainingPackage.dataprocessing.basic_pre_processing import filterCardinality, quantileTransform, truncateData ,balancedTruncateData, oneHotEncodePredictors, oneHotEncodeTargets, toDataFrame, splitTrainValTest, balancedSplitTrainValTest, labelEncodeTargets\n\nn_sample = 20000\nsplit = [0.5, 0.25, 0.25]\nquantile_transform_distribution='normal'\n\nexclude_models = ('CatBoost_reg', 'CatBoost_class')\n\n\ndata_pre_processing = dataPreProcessingEventEmitter()\n\nfilter_cardinality = filterCardinality(transform = 'all')\ntruncate_data = truncateData(n = n_sample, transform = 'all')\nbalanced_truncate